# 🛒 Sales & Customer Behaviour Insights – Green Cart Ltd.
**Analysis using Jupyter (Python) and SQL**

This notebook explores sales and customer behaviour for Green Cart Ltd., a UK-based eco-friendly e-commerce company. The goal is to provide actionable insights to support marketing and operational decisions.

## 📥 Task 1: Load the Data

We begin by importing the necessary libraries and loading the three datasets into separate DataFrames.

In [ ]:
import pandas as pd
import numpy as np

sales_df = pd.read_csv("sales_data.csv")
product_df = pd.read_csv("product_info.csv")
customer_df = pd.read_csv("customer_info.csv")

print("✅ Data loaded successfully")

## 🧹 Task 2: Clean the Data

Data cleaning includes standardising text format, converting date columns, handling missing values, and removing duplicates.

In [ ]:
# Standardise text formatting
sales_df['delivery_status'] = sales_df['delivery_status'].str.strip().str.title()
sales_df['payment_method'] = sales_df['payment_method'].str.strip().str.title()
sales_df['region'] = sales_df['region'].str.strip().str.title()

customer_df['gender'] = customer_df['gender'].str.strip().str.title()
customer_df['region'] = customer_df['region'].str.strip().str.title()
customer_df['loyalty_tier'] = customer_df['loyalty_tier'].str.strip().str.title()

# Convert dates
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'], errors='coerce')
product_df['launch_date'] = pd.to_datetime(product_df['launch_date'], errors='coerce')
customer_df['signup_date'] = pd.to_datetime(customer_df['signup_date'], errors='coerce')

# Handle missing values
sales_df['discount_applied'] = sales_df['discount_applied'].fillna(0.0)
sales_df = sales_df.dropna(subset=['order_id', 'customer_id', 'product_id', 'order_date'])

customer_df['loyalty_tier'] = customer_df['loyalty_tier'].fillna('Unknown')
customer_df['region'] = customer_df['region'].fillna('Unknown')
customer_df['email'] = customer_df['email'].fillna('unknown@unknown.com')

# Remove duplicates
sales_df = sales_df.drop_duplicates(subset=['order_id'])
product_df = product_df.drop_duplicates(subset=['product_id'])
customer_df = customer_df.drop_duplicates(subset=['customer_id'])

# Validate numeric fields
sales_df['quantity'] = pd.to_numeric(sales_df['quantity'], errors='coerce')
sales_df = sales_df[sales_df['quantity'] >= 0]
sales_df = sales_df[sales_df['unit_price'] >= 0]
sales_df = sales_df[sales_df['discount_applied'].between(0, 1)]

print("✅ Data cleaned and standardised")

## 🔗 Task 3: Merge the Data

We now merge the three cleaned DataFrames to create a unified view of the sales, customer, and product data.

In [ ]:
# Merge sales with products, then with customers
merged_df = sales_df.merge(product_df, on='product_id', how='left')
merged_df = merged_df.merge(customer_df, on='customer_id', how='left')

# Quick check
print("✅ Merged data shape:", merged_df.shape)
merged_df.head()

## 🧠 Task 4: Feature Engineering

We derive new features to enrich the analysis and extract business insights. These include revenue, price bands, and customer email domains.

In [ ]:
# Create new features
merged_df['revenue'] = merged_df['quantity'] * merged_df['unit_price'] * (1 - merged_df['discount_applied'])
merged_df['order_week'] = merged_df['order_date'].dt.isocalendar().week
merged_df['price_band'] = pd.cut(
    merged_df['unit_price'],
    bins=[0, 15, 30, float('inf')],
    labels=['Low', 'Medium', 'High']
)
merged_df['days_to_order'] = (merged_df['order_date'] - merged_df['launch_date']).dt.days
merged_df['email_domain'] = merged_df['email'].str.extract(r'@(.+)$')
merged_df['is_late'] = merged_df['delivery_status'] == "Delayed"

# Preview engineered dataset
merged_df[['revenue', 'order_week', 'price_band', 'days_to_order', 'email_domain', 'is_late']].head()